In [9]:
import copy  # tensor copy용
import numpy as np # numpy로만 rnn구현

In [22]:
def sigmoid(x) :
    output = 1 / (1+np.exp(-x))
    return output

In [23]:
def sigmoid_output_to_derivative(output) :
    return output*(1-output)

In [39]:
sigmoid_output_to_derivative(-90)

-8190

In [26]:
int2binary = {}
binary_dim = 8

largest_number = pow(2, binary_dim)

largest_number

256

In [28]:

binary = np.unpackbits(
    np.array([range(largest_number)], dtype=np.uint8).T, axis=1)
binary

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       ..., 
       [1, 1, 1, ..., 1, 0, 1],
       [1, 1, 1, ..., 1, 1, 0],
       [1, 1, 1, ..., 1, 1, 1]], dtype=uint8)

In [29]:
for i in range(largest_number) :
    int2binary[i] = binary[i]

In [31]:
# input variables
alpha = 0.1 #학습률
input_dim = 2  # a+ b-> input value
hidden_dim = 16 # hidden layer 16
output_dim = 1  # a+b = c -> 1

In [32]:
# initialize neural network weights
synapse_0 = 2 * np.random.random((input_dim, hidden_dim)) -1
synapse_1 = 2 * np.random.random((hidden_dim, output_dim)) - 1
synapse_h = 2 * np.random.random((hidden_dim, hidden_dim)) - 1
print(synapse_0.shape)
print(synapse_1.shape)
print(synapse_h.shape)

(2, 16)
(16, 1)
(16, 16)


In [37]:
pow(0.9,50)

0.00515377520732012

In [ ]:
sigmoid_output_to_derivative()

In [ ]:


#별도로 시냅스 업데이트 정보를 임시로 저장하기 위한 변수
synapse_0_update = np.zeros_like(synapse_0)
synapse_1_update = np.zeros_like(synapse_1)
synapse_h_update = np.zeros_like(synapse_h)

![](./data/layers.png)

In [8]:
# 1만번 학습
for j in range(10000):

    # a + b = c   
    # a,b는 random 생성
    a_int = np.random.randint(largest_number/2)
    a = int2binary[a_int]

    b_int = np.random.randint(largest_number/2)
    b = int2binary[b_int]

    # 실제 결과값
    c_int = a_int + b_int
    c = int2binary[c_int]

    # prediction value 저장 변수
    d = np.zeros_like(c)

    overallError = 0

    # layer1 ( input -> hidden layer )
    layer_1_values = list()  # single value
    # 초기엔 이전 상태의 hidden layer가 없기 때문에 0으로 초기화된 array를 별도로 저장
    layer_1_values.append(np.zeros(hidden_dim)) # list of above
    # layer2의 back props value를 보관
    layer_2_deltas = list()

    # forward propagation
    for position in range(binary_dim) :

        # genearte input and output Transpose 행렬 취해서 결과 포멧shape(1, 8)을 맞춰줌 
        X = np.array( [[ a[binary_dim - position - 1], b[binary_dim - position - 1]]])
        y = np.array( [[ c[binary_dim - position - 1]]]).T

        # hidden layer (input ~+ prev_hidden)  그림1
        layer_1 = sigmoid( np.dot(X, synapse_0) + np.dot(layer_1_values[-1], synapse_h) )

        # output layer 여기는 post_hidden layer ->  output layer
        layer_2 = sigmoid(np.dot(layer_1, synapse_1))

        # 결과값과의 차이
        layer_2_error = y - layer_2 
        
        # 결과값의 차이를 sigmoid
        # layer 2의 델타 값은 sigmoid미분값(잔차)에 layer2_error를 곱함 (back props시 변경되어야 하는 변화량)
        layer_2_deltas.append( (layer_2_error) * sigmoid_output_to_derivative(layer_2)) 
        #  overallError변수에 계속 델타값을 누적
        # 여기에서 왜 리스트를 취하는걸까? single value인데 말이다 그리고 절대값을 취하는건 건 왜 인지 모르겠음..
        overallError += np.abs(layer_2_error[0]) 

        # 예측값 저장
        d[binary_dim - position -1 ] = np.round(layer_2[0][0]) # 0.5는 1인줄 알았더니 0으로 처리함

        # RNN이므로 현재 상태의 hidden layer의 값을 다음 상태의 hidden layer에 input으로 사용하기 위해 저장
        layer_1_values.append(copy.deepcopy(layer_1))

    # 이 변수는 back prop시에도 마찬가지로 마지막 다음의 hidden layer를 별도로 상태
    future_layer_1_delta  = np.zeros(hidden_dim)

    # backpropagation 진행
    for position in range(binary_dim) :
        
        X = np.array([[a[position], b[position]]])
        layer_1 = layer_1_values[-position-1] # 이전 layer1의 값
        prev_layer_1= layer_1_values[-position-2] # 그 이전의 layer1의 값

        # error of output layer
        layer_2_delta = layer_2_deltas[-position-1] # layer2의 delta도 가져옴

        # error of hidden layer
        layer_1_delta = (  future_layer_1_delta.dot(synapse_h.T)
                            + layer_2_delta.dot(synapse_1.T)
                        ) * sigmoid_output_to_derivative(layer_1)

        # delta값을 이용하여 시냅스의 weight를 조정하는 값을 계산
        synapse_1_update += np.atleast_2d(layer_1).T.dot(layer_2_delta)
        synapse_h_update += np.atleast_2d(prev_layer_1).T.dot(layer_1_delta)
        synapse_0_update += X.T.dot(layer_1_delta)

        # 현재 상태의 이용한 delta값을 저장 
        future_layer_1_delta = layer_1_delta

    # alpha는 학습률이며 현재 시냅스를 실제 조정하는 부분
    synapse_0 += synapse_0_update * alpha
    synapse_1 += synapse_1_update * alpha
    synapse_h += synapse_h_update * alpha

    # 초기화
    synapse_0_update *= 0
    synapse_1_update *= 0
    synapse_h_update *= 0

    # 1000번 돌때마다 overallError를 계산
    if (j % 1000 == 0):
        print("Error:" + str(overallError))
        print("Pred:" + str(d))
        print("True:" + str(c))
        out = 0
        for index, x in enumerate(reversed(d)):
            out += x * pow(2, index)
        print(str(a_int) + " + " + str(b_int) + " = " + str(out))
        print("------------")





Error:[ 4.88375701]
Pred:[1 1 1 1 1 1 1 1]
True:[1 0 0 0 0 0 0 1]
52 + 77 = 255
------------
Error:[ 3.98479372]
Pred:[1 1 0 0 0 0 0 0]
True:[1 0 1 0 1 0 1 0]
66 + 104 = 192
------------
Error:[ 3.60347871]
Pred:[0 0 1 1 1 1 1 1]
True:[0 0 0 1 1 1 1 1]
23 + 8 = 63
------------
Error:[ 3.63916913]
Pred:[1 1 0 1 1 0 1 1]
True:[1 0 0 1 1 0 1 1]
46 + 109 = 219
------------
Error:[ 2.89716074]
Pred:[1 0 1 1 1 1 0 1]
True:[1 0 1 1 1 1 0 1]
110 + 79 = 189
------------
Error:[ 2.27077671]
Pred:[1 0 1 0 0 1 0 0]
True:[1 0 1 0 0 1 0 0]
122 + 42 = 164
------------
Error:[ 0.85480656]
Pred:[0 0 0 1 1 1 1 1]
True:[0 0 0 1 1 1 1 1]
25 + 6 = 31
------------
Error:[ 1.21367149]
Pred:[1 0 0 0 1 0 0 0]
True:[1 0 0 0 1 0 0 0]
28 + 108 = 136
------------
Error:[ 0.56660859]
Pred:[1 0 0 1 1 0 1 1]
True:[1 0 0 1 1 0 1 1]
67 + 88 = 155
------------
Error:[ 0.55687411]
Pred:[1 1 1 1 0 1 0 1]
True:[1 1 1 1 0 1 0 1]
121 + 124 = 245
------------
